In [12]:
import pandas as pd
from pycaret.time_series import *
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../data/train.csv')
train.head(3)

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818


In [5]:
prod0 = train.loc[(train['product'] == 0) & (train['store'] == 0)].copy()
prod0.head(3)

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818


In [6]:
prod0.drop(columns=['product', 'store'], inplace=True)
prod0['Date'] = pd.to_datetime(prod0['Date'])
prod0.rename(columns={'Date': 'ds', 'number_sold': 'y'}, inplace=True)
prod0.set_index('ds', inplace=True)
prod0.head(3)

,y
ds,
2010-01-01,801
2010-01-02,810
2010-01-03,818


## Modeling

In [ ]:
s = setup(
    data=prod0['y'],
    fh = 365,
    fold=3,
    seasonal_period=365
)

best_model = compare_models()

,Description,Value
0,session_id,5430
1,Target,y
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(3287, 1)"
5,Transformed data shape,"(3287, 1)"
6,Transformed train set shape,"(2922, 1)"
7,Transformed test set shape,"(365, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,2.9808,2.6977,33.1383,37.7308,0.0399,0.0399,0.0338,8.2600
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,3.0333,3.1424,33.7459,44.0014,0.0396,0.0409,-0.4777,3.2633
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,3.0349,2.7503,33.7395,38.4670,0.0406,0.0406,-0.0041,3.1267
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,3.0349,2.7503,33.7395,38.4670,0.0406,0.0406,-0.0041,4.9733
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,3.0349,2.7503,33.7395,38.4670,0.0406,0.0406,-0.0041,4.0567
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,3.0349,2.7503,33.7398,38.4673,0.0406,0.0406,-0.0041,3.4867
arima,ARIMA,3.0350,2.7538,33.7414,38.5162,0.0405,0.0406,-0.0067,0.1400
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,3.0352,2.7505,33.7431,38.4701,0.0406,0.0407,-0.0043,3.1400
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,3.0352,2.7505,33.7430,38.4700,0.0406,0.0407,-0.0042,4.0900
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,3.0352,2.7505,33.7431,38.4701,0.0406,0.0407,-0.0043,3.0633


c:\Users\User\anaconda3\envs\prophet_env\lib\site-packages\pycaret\internal\pycaret_experiment\supervised_experiment.py:713: UserWarning:

Unsupported estimator `ensemble_forecaster` for method `compare_models()`, removing from model_library

c:\Users\User\anaconda3\envs\prophet_env\lib\site-packages\pycaret\time_series\forecasting\oop.py:2694: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

c:\Users\User\anaconda3\envs\prophet_env\lib\site-packages\pycaret\time_series\forecasting\oop.py:2694: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rele

In [8]:
best_model

AutoARIMA(random_state=5430, suppress_warnings=True)

In [14]:
final_model = finalize_model(best_model)
future_predictions = predict_model(final_model, fh=365)
future_predictions

,y_pred
2019-01-01,840.9415
2019-01-02,841.2852
2019-01-03,841.1461
2019-01-04,841.1054
2019-01-05,841.0649
...,...
2019-12-27,832.2905
2019-12-28,832.2769
2019-12-29,832.2634
2019-12-30,832.2499


In [15]:
plt.figure(figsize=(15,5))
plot_model(final_model, plot='forecast')
plt.show()

<Figure size 1500x500 with 0 Axes>

## Evaluation

In [16]:
test = pd.read_csv('../data/test data.csv')
test = test.loc[(test['product']==0)&(test['store']==0)]
test.drop(columns=['product', 'store'], inplace=True)
test

,Date,number_sold
0,2019-01-01,845
1,2019-01-02,851
2,2019-01-03,840
3,2019-01-04,842
4,2019-01-05,845
...,...,...
360,2019-12-27,848
361,2019-12-28,856
362,2019-12-29,855
363,2019-12-30,862


In [17]:
from statsmodels.tools.eval_measures import rmse
print('RMSE:', rmse(test['number_sold'], future_predictions['y_pred']))

RMSE: 36.871568272541815
